In [58]:
from datetime import datetime
import plotly.graph_objects as go
import pandas as pd
import numpy as np

In [59]:
data = pd.read_csv('assets/models/jeff_multi_factor/aggregate_features.csv', index_col=0)
data = data.drop(['Unnamed: 0_x','Unnamed: 0_y','close',], axis=1)

c:\users\windows\pycharmprojects\siads697\test\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



In [60]:
data = data.sort_values(['Instrument', 'date'])
data['date'] = pd.to_datetime(data['date'])
date = datetime.strptime('2019-07-31','%Y-%m-%d')
reduced = data.dropna(subset=['12m_volume'], axis = 0)

In [61]:
labels = reduced.filter(['date','Instrument','1yr_ret', '3m_ret', '6m_ret'])
features = reduced.drop(['1yr_ret', '3m_ret', '6m_ret'], axis = 1)

In [62]:
corr_dat = reduced.drop(['3m_ret', '6m_ret'], axis = 1).dropna()
corr_dat = corr_dat.groupby('date').corr()[['1yr_ret']].reset_index()
corr_dat = corr_dat[corr_dat['level_1']!='1yr_ret']

In [63]:
x_dat = corr_dat['level_1'].unique()[0:10]
y_dat = [list(corr_dat[corr_dat['level_1']==i]['1yr_ret']) for i in x_dat][0:10]
colors = ['rgb(136, 204, 238)', 'rgb(204, 102, 119)', 'rgb(221, 204, 119)', 'rgb(17, 119, 51)', 'rgb(51, 34, 136)',
          'rgb(170, 68, 153)', 'rgb(68, 170, 153)', 'rgb(153, 153, 51)', 'rgb(136, 34, 85)', 'rgb(102, 17, 0)',
          'rgb(136, 136, 136)']
fig = go.Figure()
for x, y, cl in zip(x_dat, y_dat, colors[0:10]):
    fig.add_trace(go.Box(
        y = y,
        name=x,
        #boxpoints='all',
        whiskerwidth= 0.2,
        line_width=1,
        fillcolor=cl
    ))
fig.update_layout(
    title='Correlation to 1 Year forward return (Features 1-10)',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=5,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)

fig.show()

In [73]:
x_dat = corr_dat['level_1'].unique()[10:20]
y_dat = [list(corr_dat[corr_dat['level_1']==i]['1yr_ret']) for i in x_dat]
colors = ['rgb(136, 204, 238)', 'rgb(204, 102, 119)', 'rgb(221, 204, 119)', 'rgb(17, 119, 51)', 'rgb(51, 34, 136)',
          'rgb(170, 68, 153)', 'rgb(68, 170, 153)', 'rgb(153, 153, 51)', 'rgb(136, 34, 85)', 'rgb(102, 17, 0)',
          'rgb(136, 136, 136)']

fig_2 = go.Figure()
for x, y, cl in zip(x_dat, y_dat, colors[0:10]):
    fig_2.add_trace(go.Box(
        y = y,
        name=x,
        #boxpoints='all',
        whiskerwidth= 0.2,
        line_width=1,
        fillcolor=cl
    ))
fig_2.update_layout(
    title='Correlation to 1 Year forward return (Features 10-20)',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=5,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)

fig_2.show()

In [75]:
x_dat = corr_dat['level_1'].unique()[20:30]
y_dat = [list(corr_dat[corr_dat['level_1']==i]['1yr_ret']) for i in x_dat]
colors = ['rgb(136, 204, 238)', 'rgb(204, 102, 119)', 'rgb(221, 204, 119)', 'rgb(17, 119, 51)', 'rgb(51, 34, 136)',
          'rgb(170, 68, 153)', 'rgb(68, 170, 153)', 'rgb(153, 153, 51)', 'rgb(136, 34, 85)', 'rgb(102, 17, 0)',
          'rgb(136, 136, 136)']

fig_3 = go.Figure()
for x, y, cl in zip(x_dat, y_dat, colors[0:10]):
    fig_3.add_trace(go.Box(
        y = y,
        name=x,
        #boxpoints='all',
        whiskerwidth= 0.2,
        line_width=1,
        fillcolor=cl
    ))
fig_3.update_layout(
    title='Correlation to 1 Year forward return (Features 20-30)',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=5,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)

fig_3.show()

In [77]:
x_dat = corr_dat['level_1'].unique()[30:40]
y_dat = [list(corr_dat[corr_dat['level_1']==i]['1yr_ret']) for i in x_dat]
colors = ['rgb(136, 204, 238)', 'rgb(204, 102, 119)', 'rgb(221, 204, 119)', 'rgb(17, 119, 51)', 'rgb(51, 34, 136)',
          'rgb(170, 68, 153)', 'rgb(68, 170, 153)', 'rgb(153, 153, 51)', 'rgb(136, 34, 85)', 'rgb(102, 17, 0)',
          'rgb(136, 136, 136)']

fig_4 = go.Figure()
for x, y, cl in zip(x_dat, y_dat, colors[0:10]):
    fig_4.add_trace(go.Box(
        y = y,
        name=x,
        #boxpoints='all',
        whiskerwidth= 0.2,
        line_width=1,
        fillcolor=cl
    ))
fig_4.update_layout(
    title='Correlation to 1 Year forward return (Features 30-40)',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=5,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)

fig_4.show()


In [80]:
x_dat = corr_dat['level_1'].unique()[40:50]
y_dat = [list(corr_dat[corr_dat['level_1']==i]['1yr_ret']) for i in x_dat]
colors = ['rgb(136, 204, 238)', 'rgb(204, 102, 119)', 'rgb(221, 204, 119)', 'rgb(17, 119, 51)', 'rgb(51, 34, 136)',
          'rgb(170, 68, 153)', 'rgb(68, 170, 153)', 'rgb(153, 153, 51)', 'rgb(136, 34, 85)', 'rgb(102, 17, 0)',
          'rgb(136, 136, 136)']

fig_5 = go.Figure()
for x, y, cl in zip(x_dat, y_dat, colors[0:10]):
    fig_5.add_trace(go.Box(
        y = y,
        name=x,
        #boxpoints='all',
        whiskerwidth= 0.2,
        line_width=1,
        fillcolor=cl
    ))
fig_5.update_layout(
    title='Correlation to 1 Year forward return (Features 40-50)',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=5,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)

fig_5.show()

In [81]:
x_dat = corr_dat['level_1'].unique()[50:60]
y_dat = [list(corr_dat[corr_dat['level_1']==i]['1yr_ret']) for i in x_dat]
colors = ['rgb(136, 204, 238)', 'rgb(204, 102, 119)', 'rgb(221, 204, 119)', 'rgb(17, 119, 51)', 'rgb(51, 34, 136)',
          'rgb(170, 68, 153)', 'rgb(68, 170, 153)', 'rgb(153, 153, 51)', 'rgb(136, 34, 85)', 'rgb(102, 17, 0)',
          'rgb(136, 136, 136)']

fig_6 = go.Figure()
for x, y, cl in zip(x_dat, y_dat, colors[0:10]):
    fig_6.add_trace(go.Box(
        y = y,
        name=x,
        #boxpoints='all',
        whiskerwidth= 0.2,
        line_width=1,
        fillcolor=cl
    ))
fig_6.update_layout(
    title='Correlation to 1 Year forward return (Features 40-50)',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=5,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)

fig_6.show()


In [82]:
x_dat = corr_dat['level_1'].unique()[60:]
y_dat = [list(corr_dat[corr_dat['level_1']==i]['1yr_ret']) for i in x_dat]
colors = ['rgb(136, 204, 238)', 'rgb(204, 102, 119)', 'rgb(221, 204, 119)', 'rgb(17, 119, 51)', 'rgb(51, 34, 136)',
          'rgb(170, 68, 153)', 'rgb(68, 170, 153)', 'rgb(153, 153, 51)', 'rgb(136, 34, 85)', 'rgb(102, 17, 0)',
          'rgb(136, 136, 136)']

fig_7 = go.Figure()
for x, y, cl in zip(x_dat, y_dat, colors[0:10]):
    fig_7.add_trace(go.Box(
        y = y,
        name=x,
        #boxpoints='all',
        whiskerwidth= 0.2,
        line_width=1,
        fillcolor=cl
    ))
fig_7.update_layout(
    title='Correlation to 1 Year forward return (Features 40-50)',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=5,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)

fig_7.show()

